In [3]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('dataset/hell-week-mic/train.csv')
df.head()

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state
0,T77962103,7059868985,673.0,2.131868e+14,Copy,guerrerodavid@example.org,http://garcia.org/,Gabon,📑,Cathy Cherry,...,🍣🏿🟠U👨@9.028568970658524@️bY‍t,N6♀rs%-0.17374199145886737%g2Fu7,dwLNC+-16.438253859793118+D🟠Mj👩,♀Y‍tt^-5.683708646092431^7b🎓🍣T,Q👩👨H🏾?4.495020766798911?p🎨dBV,JC🍣8Q!1.3976340938324077!Wo‍6t,QtgcM|-2.1616012233562616|srwW❇,♀👩🦐pq%7.619063527183258%👨Oet♀,‍🏿🟠👩X+-7.00274740118153+‍b🎨VK,Despair
1,O92591533,+1-288-810-2425x013,983.0,4.820055e+15,Interior and spatial designer,michaela47@example.org,http://ortiz.com/,Bermuda,🥷🏿,Austin Graham,...,🍣🏿🟠U👨@-40.77392373961654@️bY‍t,N6♀rs%3.836527507653148%g2Fu7,dwLNC+-20.503862282922828+D🟠Mj👩,♀Y‍tt^4.345119531921794^7b🎓🍣T,Q👩👨H🏾?-0.19904265803833332?p🎨dBV,JC🍣8Q!3.414072326617339!Wo‍6t,QtgcM|8.673900995107994|srwW❇,♀👩🦐pq%-0.6359934104694338%👨Oet♀,‍🏿🟠👩X+-11.039355040136286+‍b🎨VK,Worthlessness
2,O31883571,791.502.4387x7276,855.0,4.031429e+15,"Scientist, water quality",zhoffman@example.org,http://kelly.com/,Sao Tome and Principe,👩‍💻,Frank Smith,...,🍣🏿🟠U👨@32.83118091536586@️bY‍t,N6♀rs%6.673810456792015%g2Fu7,dwLNC+31.545003675536307+D🟠Mj👩,♀Y‍tt^-1.1189348865153141^7b🎓🍣T,Q👩👨H🏾?-2.109315804181183?p🎨dBV,JC🍣8Q!6.773526691273227!Wo‍6t,QtgcM|1.7184076239046218|srwW❇,♀👩🦐pq%-35.95525145090523%👨Oet♀,‍🏿🟠👩X+8.6752786215897+‍b🎨VK,Stress
3,M18080565,2522308761,806.0,3.036423e+13,Toxicologist,hprice@example.net,http://mann.info/,Korea,NaN,Austin Young,...,🍣🏿🟠U👨@-11.829454766573882@️bY‍t,N6♀rs%-10.87012476653127%g2Fu7,dwLNC+26.131133464440587+D🟠Mj👩,♀Y‍tt^2.1159115210320385^7b🎓🍣T,Q👩👨H🏾?7.373067390546165?p🎨dBV,JC🍣8Q!-0.2388097363537749!Wo‍6t,QtgcM|-5.42787813792933|srwW❇,♀👩🦐pq%24.501780969213996%👨Oet♀,‍🏿🟠👩X+-0.8456272162320895+‍b🎨VK,Fatigue
4,Q74073854,683-521-2001x423,424.0,4.610207e+15,Software engineer,michael61@example.net,https://www.mitchell.biz/,Russian Federation,💆🏿‍♂️,Kimberly Wilson,...,🍣🏿🟠U👨@10.957430264267883@️bY‍t,N6♀rs%-1.5309050040841568%g2Fu7,dwLNC+-17.670060957130605+D🟠Mj👩,♀Y‍tt^0.132418797411758^7b🎓🍣T,Q👩👨H🏾?3.6960467337932292?p🎨dBV,JC🍣8Q!4.588960052905303!Wo‍6t,QtgcM|-3.735829327461521|srwW❇,♀👩🦐pq%18.194146730985878%👨Oet♀,‍🏿🟠👩X+10.404818391481196+‍b🎨VK,Tears


In [8]:
df.shape

(140000, 111)